## Import libraries 

In [37]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from joblib import dump, load
import warnings
import cvxpy as cp
from sklearn.covariance import LedoitWolf

warnings.filterwarnings("ignore")


In [8]:
TARGET_COL = "monthly_log_return_future"
FEATURE_COLS = [
    "volume_log", "log_daily_vol_21", "log_monthly_var_3",
    "high_low_spread", "open_close_spread",
    "close_sma_ratio_20", "close_ema_ratio_20",
    "close_sma_ratio_50", "close_ema_ratio_50",
    "close_sma_ratio_100", "close_ema_ratio_100",
    "macd_line_pct", "macd_signal_pct", "macd_hist_pct",
    "daily_log_return", "daily_volume_return", "Dividends"
]
RNG = 42
N_SPLITS_CV = 5
SAVE_DIR = "linear_ticker_models"
os.makedirs(SAVE_DIR, exist_ok=True)


In [39]:
# Hyperparameter Grids for CV Models
ALPHA_GRID = np.logspace(-4, 0, 20)      # 20 alpha values from 0.0001 to 1.0
L1_RATIOS = [0.1, 0.5, 0.7, 0.9, 0.95, 1.0] # L1 ratios for Elastic Net (1.0 is pure Lasso)

TRAIN_WINDOW = 252       # 1 year lookback (trading days) for CV
TEST_WINDOW = 21         # 1 month ahead (trading days)
PURGE_DAYS = 21          # Days to remove from train to avoid leakage (H)

# --- Backtesting Constants ---
LOOKBACK = 252           # days for daily μ, Σ estimation
HORIZON = 21             # holding period (trading days)
STEP = 21                # form a new, independent portfolio every 21 days
GAMMA = 5.0              # Markowitz risk aversion parameter
NAME_CAP = 0.10          # upper limit for each stock’s weight
COST_BPS_ONE_WAY = 5.0   # linear cost per side, in bps of traded notional
MIN_NONMISSING_PCT = 0.50 # require >= 50% non-missing in lookback


###  Markowitz Backtesting Helper Functions

In [41]:
def ledoit_wolf_cov(rets_values):
    """Estimates covariance matrix using the Ledoit-Wolf shrinkage method."""
    lw = LedoitWolf()
    lw.fit(rets_values)
    # Scale daily covariance up to the 21-day horizon
    return lw.covariance_ 

def ticker_period_compound(rets_df: pd.DataFrame, start_idx: int, horizon: int):
    """Calculates the simple compound return over the HORIZON for a single leg."""
    period_returns = rets_df.iloc[start_idx + 1 : start_idx + 1 + horizon]
    gross_returns = 1.0 + period_returns
    # Missing daily returns are treated as no return (1.0 gross return)
    compound_return = gross_returns.fillna(1.0).prod(axis=0) - 1.0
    return compound_return

def solve_markowitz_long_only(mu_vector, Sigma_matrix, gamma, name_cap=None):
    """Solves the Markowitz optimization problem using CVXPY."""
    n = len(mu_vector)
    w = cp.Variable(n)
    
    # Objective: Minimize 0.5 * w'Σw - gamma * μ'w
    objective = cp.Minimize(cp.quad_form(w, Sigma_matrix) * 0.5 - gamma * mu_vector @ w)
    
    constraints = [
        cp.sum(w) == 1, 
        w >= 0          
    ]
    
    if name_cap is not None:
        constraints.append(w <= name_cap)
        
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.ECOS) 

    if problem.status not in ["optimal", "optimal_inaccurate"]:
        raise RuntimeError(f"Optimization failed with status: {problem.status}")
        
    return w.value

def backtest_markowitz_from_mu(rets: pd.DataFrame, is_in_sp500: pd.DataFrame, mu21_hat: pd.DataFrame):
    """Performs Markowitz backtest using ML-predicted expected returns (mu21_hat)."""
    rets = rets.sort_index().dropna(how="all")
    is_in_sp500 = is_in_sp500.reindex_like(rets).fillna(0).astype(int)
    mu21_hat = mu21_hat.reindex(index=rets.index, columns=rets.columns)

    dates = rets.index
    start = LOOKBACK - 1
    end = len(dates) - HORIZON - 1
    legs = []

    for t in range(start, end + 1, STEP):
        date_t = dates[t]
        
        # 1. Universe Selection
        tickers = is_in_sp500.columns[is_in_sp500.loc[date_t] == 1].tolist()
        if not tickers: continue

        # 2. Lookback data for covariance
        win = rets.iloc[t-LOOKBACK+1 : t+1][tickers]
        
        # Quality filter
        non_missing_ratio = 1.0 - win.isna().mean()
        keep = non_missing_ratio[non_missing_ratio >= MIN_NONMISSING_PCT].index.tolist()
        win = win[keep].dropna(how="all").dropna(how="any")
        if len(win) < 30 or len(win.columns) < 2: continue
        tickers = win.columns.tolist()

        # 3. Predicted Returns
        mu_row = mu21_hat.loc[date_t, tickers].astype(float).dropna()
        if len(mu_row) < 2: continue
        
        win = win[mu_row.index.tolist()] # Ensure alignment
        tickers = mu_row.index.tolist()

        # 4. Covariance Estimation
        Sigma_d = ledoit_wolf_cov(win.values)
        Sigma_21 = HORIZON * Sigma_d
        
        # 5. Optimization
        try:
            w = solve_markowitz_long_only(mu_row.values, Sigma_21, gamma=GAMMA, name_cap=NAME_CAP)
        except RuntimeError:
            continue

        # 6. Realized Return & Costs
        comp_rets = ticker_period_compound(rets[tickers], start_idx=t, horizon=HORIZON)
        gross_leg_ret = float(np.dot(w, comp_rets.values))
        cost_frac = (COST_BPS_ONE_WAY * 1e-4) * 2.0
        net_leg_ret = gross_leg_ret - cost_frac

        legs.append({
            "formation_date": date_t, "n_names": len(tickers), "gross_21d_ret": gross_leg_ret, 
            "net_21d_ret": net_leg_ret, "roundtrip_cost_frac": cost_frac
        })

    legs_df = pd.DataFrame(legs).set_index("formation_date")

    # summary statistics
    summary = {}
    if not legs_df.empty:
        legs_per_year = 252.0 / HORIZON
        avg_net = legs_df["net_21d_ret"].mean()
        std_net = legs_df["net_21d_ret"].std(ddof=1)
        
        # Annualized return from compounding the average leg return
        ann_return = (1.0 + avg_net)**legs_per_year - 1.0
        # Annualized volatility from scaling the leg standard deviation
        ann_vol = std_net * np.sqrt(legs_per_year)
        
        sharpe = ann_return / ann_vol if ann_vol > 0 else np.nan
        
        summary = {
            "legs": int(len(legs_df)),
            "avg_net_21d_return": float(avg_net),
            "std_net_21d_return": float(std_net),
            "annual_return": float(ann_return), # <--- **CALCULATION ADDED HERE**
            "annual_vol": float(ann_vol),       # <--- **CALCULATION ADDED HERE**
            "sharpe": float(sharpe),
            "cost_bps_one_way": COST_BPS_ONE_WAY,
            "gamma": GAMMA,
            "name_cap": NAME_CAP
        }
    return legs_df, summary


## Model Training and Prediction Function

In [16]:
def train_and_predict_linear_model(model_name: str, model_class, train_df, test_df, covid_test, **model_params):
    """
    Trains a linear model (LassoCV, RidgeCV, ElasticNetCV), makes predictions, 
    and returns performance metrics and prediction DataFrames.
    
    This version correctly handles model-specific parameters like 'max_iter', 
    'n_jobs', and 'random_state'.
    """
    print(f"\n--- Starting Training for {model_name} ---")
    
    results_all = []
    pred_rows_test = []
    pred_rows_covid = []
    
    # 1. Define model-specific parameters
    tscv = TimeSeriesSplit(n_splits=N_SPLITS_CV)
    
    # Parameters shared by all models (CV object and user-provided params like 'alphas')
    base_params = {
        "cv": tscv, 
        **model_params
    }
    
    # Iterative solvers (LassoCV and ElasticNetCV) require these
    iterative_params = {
        "max_iter": 2000,
        "n_jobs": -1,
        # random_state is set *after* instantiation due to a bug in some scikit-learn versions
        # that affects how it is handled internally for CV models.
    }
    
    # Build the final instantiation parameters
    if model_name == "Ridge":
        # RidgeCV only takes 'cv' and 'alphas' (from **model_params)
        final_init_params = base_params
    else: # Lasso and ElasticNet
        final_init_params = {**base_params, **iterative_params}

    
    for ticker, df_ticker_train_full in train_df.groupby("ticker"):
        
        # Prepare Ticker Data
        df_ticker_train = df_ticker_train_full.dropna(subset=FEATURE_COLS + [TARGET_COL])
        test_ticker = test_df[test_df["ticker"] == ticker].copy().dropna(subset=FEATURE_COLS + [TARGET_COL])
        covid_ticker = covid_test[covid_test["ticker"] == ticker].copy().dropna(subset=FEATURE_COLS + [TARGET_COL])

        if len(df_ticker_train) < LOOKBACK:
            # print(f"Skipping {ticker} - not enough training data.")
            continue
            
        # 1. Data Preparation and Scaling
        X_train, y_train = df_ticker_train[FEATURE_COLS], df_ticker_train[TARGET_COL]
        scaler = StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)

        # 2. Model Training (CV handles alpha selection over TimeSeriesSplit)
        
        # Instantiate the model with correct parameters
        model = model_class(**final_init_params)
        
        # Set random_state conditionally for iterative solvers (Lasso/ElasticNet)
        if model_name in ["Lasso", "ElasticNet"]:
            model.set_params(random_state=RNG)
            
        try:
            model.fit(X_train_scaled, y_train)
        except Exception as e:
            # print(f"Error fitting {model_name} for {ticker}: {e}")
            continue
            
        # 3. Save Model and Scaler
        save_path = os.path.join(SAVE_DIR, f"{ticker}_{model_name.lower()}.joblib")
        # Check if the model has alpha_ attribute (it should for all CV models)
        best_alpha = getattr(model, 'alpha_', np.nan)
        dump({"model": model, "scaler": scaler, "best_alpha": best_alpha}, save_path)
        # print(f"Trained {model_name} for {ticker}. Best alpha: {model.alpha_:.6f}")


        # 4. Predict on Internal Test
        if not test_ticker.empty:
            X_test_scaled = scaler.transform(test_ticker[FEATURE_COLS])
            y_test = test_ticker[TARGET_COL]
            y_pred = model.predict(X_test_scaled)

            r2 = r2_score(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            results_all.append({"ticker": ticker, "split": "internal_test", "model": model_name, "r2": r2, "rmse": rmse})

            pred_rows_test.append(pd.DataFrame({
                "Date": test_ticker["Date"].values, "ticker": test_ticker["ticker"].values,
                "y_pred_log": y_pred, "y_true_log": y_test.values
            }))

        # 5. Predict on COVID Stress Test
        if not covid_ticker.empty:
            X_covid_scaled = scaler.transform(covid_ticker[FEATURE_COLS])
            y_covid = covid_ticker[TARGET_COL]
            y_pred_covid = model.predict(X_covid_scaled)
            
            r2_covid = r2_score(y_covid, y_pred_covid)
            rmse_covid = np.sqrt(mean_squared_error(y_covid, y_pred_covid))
            results_all.append({"ticker": ticker, "split": "covid_stress", "model": model_name, "r2": r2_covid, "rmse": rmse_covid})

            pred_rows_covid.append(pd.DataFrame({
                "Date": covid_ticker["Date"].values, "ticker": covid_ticker["ticker"].values,
                "y_pred_log": y_pred_covid, "y_true_log": y_covid.values
            }))

    # Compile Results and Predictions
    preds_test = pd.concat(pred_rows_test, ignore_index=True) if pred_rows_test else pd.DataFrame()
    preds_covid = pd.concat(pred_rows_covid, ignore_index=True) if pred_rows_covid else pd.DataFrame()

    if not preds_test.empty:
        preds_test["R21_pred"] = np.expm1(preds_test["y_pred_log"])
    if not preds_covid.empty:
        preds_covid["R21_pred"] = np.expm1(preds_covid["y_pred_log"])

    return pd.DataFrame(results_all), preds_test, preds_covid



## Main Execution Block

### Data Loading and Splitting

In [ ]:
try:
    train_full = pd.read_csv("../Data prep and EDA/processed_data/training_data.csv", parse_dates=["Date"])
    covid_test = pd.read_csv("../Data prep and EDA/processed_data/covid_stress_test_data.csv", parse_dates=["Date"])
except FileNotFoundError:
    print("WARNING: Data files not found at expected relative paths. Using local stubs for demonstration.")
    train_full = pd.DataFrame(columns=["ticker", "Date", TARGET_COL] + FEATURE_COLS)
    covid_test = pd.DataFrame(columns=["ticker", "Date", TARGET_COL] + FEATURE_COLS)

train_full = train_full.sort_values(["ticker", "Date"]).reset_index(drop=True)
covid_test = covid_test.sort_values(["ticker", "Date"]).reset_index(drop=True)

# Clean 'Dividends' column if necessary
for df in [train_full, covid_test]:
    if "Dividends" in df.columns:
        df["Dividends"] = pd.to_numeric(
            df["Dividends"].astype(str).str.replace("USD", "", regex=False).str.strip().replace("", np.nan),
            errors='coerce'
        )

# Define internal train/test split (last 2 years of the training data)
unique_dates = sorted(train_full["Date"].unique())
if len(unique_dates) > LOOKBACK * 2:
    split_idx = len(unique_dates) - LOOKBACK * 2
    split_date = unique_dates[split_idx]
    train_df = train_full[train_full["Date"] <= split_date].copy()
    test_df = train_full[train_full["Date"] > split_date].copy()
    
    # Purge last 21 feature dates from train, ticker-wise
    H = HORIZON
    last_train_by_ticker = train_df.groupby("ticker")["Date"].max().rename("last_train_date")
    train_df = train_df.merge(last_train_by_ticker, on="ticker")
    train_df = train_df[train_df["Date"] <= (train_df["last_train_date"] - pd.tseries.offsets.BDay(H))]
    train_df = train_df.drop(columns=["last_train_date"])

    print(f"Data split: Training up to {split_date.date()}.")
else:
    # Use full original training data as train_df, and an empty test_df if data is insufficient
    train_df = train_full.copy()
    test_df = pd.DataFrame()
    print("Warning: Insufficient data for a 2-year internal test split.")

Data split: Training up to 2018-02-16.


### Run All Models and Collect Predictions

In [22]:
model_results = {}
backtest_data = {}

# 1. Lasso Model
results_lasso, preds_test_lasso, preds_covid_lasso = train_and_predict_linear_model(
    "Lasso", LassoCV, train_df, test_df, covid_test, alphas=ALPHA_GRID.tolist()
)
model_results["Lasso"] = results_lasso
backtest_data["Lasso"] = {"test": preds_test_lasso, "covid": preds_covid_lasso}

# 2. Ridge Model
# RidgeCV only needs 'alphas' (passed via **model_params)
results_ridge, preds_test_ridge, preds_covid_ridge = train_and_predict_linear_model(
    "Ridge", RidgeCV, train_df, test_df, covid_test, alphas=ALPHA_GRID.tolist()
)
model_results["Ridge"] = results_ridge
backtest_data["Ridge"] = {"test": preds_test_ridge, "covid": preds_covid_ridge}


# 3. Elastic Net Model
results_en, preds_test_en, preds_covid_en = train_and_predict_linear_model(
    "ElasticNet", ElasticNetCV, train_df, test_df, covid_test, 
    alphas=ALPHA_GRID.tolist(), l1_ratio=L1_RATIOS
)
model_results["ElasticNet"] = results_en
backtest_data["ElasticNet"] = {"test": preds_test_en, "covid": preds_covid_en}



--- Starting Training for Lasso ---

--- Starting Training for Ridge ---

--- Starting Training for ElasticNet ---


### Backtesting

In [58]:
df_full = pd.concat([train_full, covid_test], ignore_index=True)
df_full["daily_simple_return"] = np.expm1(df_full["daily_log_return"])
rets = df_full.pivot(index="Date", columns="ticker", values="daily_simple_return").sort_index()
is_in = df_full.pivot(index="Date", columns="ticker", values="is_in_sp500").sort_index()
is_in = is_in.fillna(0).astype(float).clip(0, 1).astype(int)


In [62]:
print("\n" + "="*50)
print("AVERAGE MODEL PREDICTION PERFORMANCE (R2)")
print("="*50)
summary_r2 = pd.concat(model_results.values()).groupby(["model", "split"])["r2"].mean().unstack()
print(summary_r2)

print("\n" + "="*50)
print("MARKOWITZ BACKTEST RESULTS)")
print("="*50)

# Initialize a list to hold all results dictionaries
all_backtest_results = [] 

for model_name, preds in backtest_data.items():
    
    # 1. Internal Test Backtest
    mu21_hat_internal = preds["test"].pivot(index="Date", columns="ticker", values="R21_pred").sort_index()
    if not mu21_hat_internal.empty:
        ml_start = mu21_hat_internal.index.min()
        
        # Define the daily return window required for lookback and holding period
        rets_ml = rets.loc[rets.index >= (ml_start - pd.tseries.offsets.BDay(LOOKBACK + HORIZON))]
        is_in_ml = is_in.loc[is_in.index >= (ml_start - pd.tseries.offsets.BDay(LOOKBACK + HORIZON))]
        
        _, summary_ml = backtest_markowitz_from_mu(rets_ml, is_in_ml, mu21_hat_internal)
        summary_ml['Model'] = model_name
        summary_ml['Test Split'] = 'Internal'
        all_backtest_results.append(summary_ml) 
    else:
        all_backtest_results.append({'Model': model_name, 'Test Split': 'Internal', 'sharpe': np.nan, 'annual_return': np.nan, 'annual_vol': np.nan, 'avg_net_21d_return': np.nan})


    # 2. COVID Stress Test Backtest
    mu21_hat_covid = preds["covid"].pivot(index="Date", columns="ticker", values="R21_pred").sort_index()
    if not mu21_hat_covid.empty:
        # Use full daily data and S&P 500 membership (rets and is_in)
        _, summary_covid = backtest_markowitz_from_mu(rets, is_in, mu21_hat_covid)
        summary_covid['Model'] = model_name
        summary_covid['Test Split'] = 'COVID'
        all_backtest_results.append(summary_covid)
    else:
        all_backtest_results.append({'Model': model_name, 'Test Split': 'COVID', 'sharpe': np.nan, 'annual_return': np.nan, 'annual_vol': np.nan, 'avg_net_21d_return': np.nan})

backtest_results_df = pd.DataFrame(all_backtest_results)
backtest_results_df = backtest_results_df.set_index(['Model', 'Test Split'])[
    ['annual_return', 'annual_vol', 'sharpe', 'avg_net_21d_return', 'legs']
].rename(columns={
    'annual_return': 'Ann. Return',
    'annual_vol': 'Ann. Volatility',
    'sharpe': 'Sharpe Ratio',
    'avg_net_21d_return': 'Avg Net 21d Ret',
    'legs': 'Legs'
})

print(backtest_results_df)

print("\nAll linear models trained and backtested.")



AVERAGE MODEL PREDICTION PERFORMANCE (R2)
split       covid_stress  internal_test
model                                  
ElasticNet     -0.042915      -0.085392
Lasso          -0.043203      -0.085192
Ridge          -0.171263      -3.636004

MARKOWITZ BACKTEST RESULTS (All Metrics)
                       Ann. Return  Ann. Volatility  Sharpe Ratio  \
Model      Test Split                                               
Lasso      Internal       0.053337         0.302269      0.176455   
           COVID          0.740931         0.483255      1.533209   
Ridge      Internal       0.227207         0.260368      0.872641   
           COVID          0.833616         0.778420      1.070907   
ElasticNet Internal       0.037898         0.299824      0.126401   
           COVID          0.687046         0.451580      1.521426   

                       Avg Net 21d Ret  Legs  
Model      Test Split                         
Lasso      Internal           0.004340    19  
           COVID     